## Setup libraries

In [33]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [34]:
Moggio = read_csv("Moggiodata14_20.csv")
#Schivenoglia = Schivenoglia.loc[:, ["Date", "Ammonia", "PM10", "PM25",
#                       "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

Moggio = Moggio.drop(["Arsenic", "Benzo_a_pyrene", "Cadmium", "Lead", "Nikel", "NO", "IDStation.1", "IDStation.2", "Date.1", "NameStation.1", "reg_Y_nn1_ID"], axis='columns')

print(Moggio.shape)
Moggio.head()



(2557, 17)


,Date,IDStation,NameStation,Ammonia,NO2,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Wind_speed_max,Wind_direction_max,Rainfall
0,2014-01-01T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,71.9,6.0,7.0,0.443554,99.0,-0.334965,45.545455,58.097902,1.805594,143.0,0.0
1,2014-01-02T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,76.8,6.0,5.0,0.355915,105.0,-0.486111,83.944444,0.000000,1.767361,153.0,7.8
2,2014-01-03T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.6,11.0,4.0,0.207631,120.0,2.220588,87.042254,30.514085,1.558451,152.0,0.0
3,2014-01-04T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,77.2,6.0,7.0,1.024926,175.0,2.137063,89.349650,0.202797,3.816783,164.0,35.8
4,2014-01-05T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.2,3.0,8.0,0.666721,94.0,2.416084,86.055944,10.951049,2.797203,136.0,25.0


# Quadrants elaboration

In [35]:
#Primo quadrante
Q1 = Moggio.loc[((Moggio.Wind_direction > 0) &
                 (Moggio.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "Date","IDStation","NameStation", "Ammonia", "NO2", "NOx", "Ozone", "PM10", "PM10", "PM25", "Wind_speed",
                 "Wind_direction",  "Temperature","Relative_humidity", "Global_radiation", "Wind_speed_max", "Wind_direction_max", "Rainfall" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Moggio.loc[((Moggio.Wind_direction > 90) &
                 (Moggio.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Moggio.loc[((Moggio.Wind_direction > 180) &
                 (Moggio.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Moggio.loc[((Moggio.Wind_direction > 270) &
                 (Moggio.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Moggio = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')


Moggio.rename(columns={'Rainfall': 'Quadrante'}, inplace=True)

Moggio

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4227/407049825.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4227/407049825.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_4227/407049825.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,Date,IDStation,NameStation,Ammonia,NO2,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Wind_speed_max,Wind_direction_max,Quadrante
0,2014-01-01T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,71.9,6.0,7.0,0.443554,-0.334965,45.545455,58.097902,1.805594,143.000000,0.0,2.0
1,2014-01-02T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,76.8,6.0,5.0,0.355915,-0.486111,83.944444,0.000000,1.767361,153.000000,7.8,2.0
2,2014-01-03T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.6,11.0,4.0,0.207631,2.220588,87.042254,30.514085,1.558451,152.000000,0.0,2.0
3,2014-01-04T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,77.2,6.0,7.0,1.024926,2.137063,89.349650,0.202797,3.816783,164.000000,35.8,2.0
4,2014-01-05T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.2,3.0,8.0,0.666721,2.416084,86.055944,10.951049,2.797203,136.000000,25.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2020-12-27T00:00:00Z,681,Moggio Loc Penscei,0.0,0.0,0.0,80.4,4.0,5.0,0.314898,1.000000,-3.706944,59.437500,50.291667,1.755556,144.0,0.0
2524,2020-12-28T00:00:00Z,681,Moggio Loc Penscei,0.0,1.9,1.4,62.3,2.0,5.0,0.345959,-3.006250,95.312500,0.618056,2.044444,171.000000,27.6,2.0
2525,2020-12-29T00:00:00Z,681,Moggio Loc Penscei,0.0,0.4,0.3,70.1,2.0,0.0,1.031443,-0.051389,88.263889,3.319444,2.523776,145.000000,2.2,2.0
2526,2020-12-30T00:00:00Z,681,Moggio Loc Penscei,0.0,1.1,1.0,63.5,0.0,0.0,0.029681,-0.549306,76.277778,14.854167,1.014679,171.000000,0.0,3.0


In [36]:
Moggio

,Date,IDStation,NameStation,Ammonia,NO2,NOx,Ozone,PM10,PM25,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Wind_speed_max,Wind_direction_max,Quadrante
0,2014-01-01T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,71.9,6.0,7.0,0.443554,-0.334965,45.545455,58.097902,1.805594,143.000000,0.0,2.0
1,2014-01-02T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,76.8,6.0,5.0,0.355915,-0.486111,83.944444,0.000000,1.767361,153.000000,7.8,2.0
2,2014-01-03T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.6,11.0,4.0,0.207631,2.220588,87.042254,30.514085,1.558451,152.000000,0.0,2.0
3,2014-01-04T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,77.2,6.0,7.0,1.024926,2.137063,89.349650,0.202797,3.816783,164.000000,35.8,2.0
4,2014-01-05T00:00:00Z,681,Moggio Loc Penscei,0.0,NaN,NaN,66.2,3.0,8.0,0.666721,2.416084,86.055944,10.951049,2.797203,136.000000,25.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2020-12-27T00:00:00Z,681,Moggio Loc Penscei,0.0,0.0,0.0,80.4,4.0,5.0,0.314898,1.000000,-3.706944,59.437500,50.291667,1.755556,144.0,0.0
2524,2020-12-28T00:00:00Z,681,Moggio Loc Penscei,0.0,1.9,1.4,62.3,2.0,5.0,0.345959,-3.006250,95.312500,0.618056,2.044444,171.000000,27.6,2.0
2525,2020-12-29T00:00:00Z,681,Moggio Loc Penscei,0.0,0.4,0.3,70.1,2.0,0.0,1.031443,-0.051389,88.263889,3.319444,2.523776,145.000000,2.2,2.0
2526,2020-12-30T00:00:00Z,681,Moggio Loc Penscei,0.0,1.1,1.0,63.5,0.0,0.0,0.029681,-0.549306,76.277778,14.854167,1.014679,171.000000,0.0,3.0


# Exporting data

In [37]:
Moggio.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Moggio/QuadrantiNuovo/QuadrantiMoggio2.csv')

